In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%env HF_HOME=out/hf_cache

In [ ]:
import networkx as nx
from absl import logging

from llm_ol.dataset import data_model
from llm_ol.experiments.post_processing import hp_search, prune_edges_out_from_node

logging.set_verbosity(logging.INFO)

In [ ]:
# G = data_model.load_graph("out/experiments/hearst/v2/graph.json")
G = data_model.load_graph("out/experiments/prompting/v5/eval/graph.json")
G_true = data_model.load_graph("out/data/wikipedia/v2/train_eval_split/test_graph.json")

# G.graph.pop("root")

In [ ]:
_, G_pruned, score = hp_search(
    G,
    G_true,
    metric="edge_f1",
    weight_threshold=[10],
    percentile_threshold=[0.9],
    merge_nodes_by_lemma=[True],
    remove_self_loops=[True],
    remove_inverse_edges=[True],
    prune_unconnected_nodes=[True],
)

In [ ]:
import random

nodes = list(G_pruned.nodes())

for _ in range(10):
    path = nx.shortest_path(G_pruned, G_pruned.graph["root"], random.choice(nodes))
    print(path[0], end="")
    for u, v in zip(path[:-1], path[1:]):
        weight = G_pruned[u][v]["weight"]
        print(f" -> ({weight}) {v}", end="")
    print()